## 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

In [5]:
#ratings 에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [6]:
#ratings 에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [7]:
#rating 과 movies 를 합친다.
data = pd.merge(movies,ratings)

In [8]:
using_cols = ['user_id','title','count']
data = data[using_cols]
data.head(10)


,user_id,title,count
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [9]:
# 가장 인기있는 영화 30개(인기순)
movie_count = data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


In [10]:
my_favorite = ['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)', 'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)']
my_movielist = pd.DataFrame({'user_id':['heemin']*5, 'title' : my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['heemin']})['user_id'].any():
    data = data.append(my_movielist)
    
data.tail(10)

,user_id,title,count
836473,5682,"Contender, The (2000)",3
836474,5812,"Contender, The (2000)",4
836475,5831,"Contender, The (2000)",3
836476,5837,"Contender, The (2000)",4
836477,5998,"Contender, The (2000)",4
0,heemin,Toy Story (1995),5
1,heemin,Jumanji (1995),5
2,heemin,Grumpier Old Men (1995),5
3,heemin,Waiting to Exhale (1995),5
4,heemin,Father of the Bride Part II (1995),5


In [13]:
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [15]:
print(user_to_idx['heemin'])
print(title_to_idx['Father of the Bride Part II (1995)'])

6039
4


In [17]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id indexing')
    data['user_id'] = temp_user_data
else:
    print('Fail')
    


user_id indexing


In [20]:
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title indexing')
    data['title'] = temp_title_data
else:
    print('Fail')
    


title indexing


In [21]:
data

,user_id,title,count
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,0,5
1,6039,1,5
2,6039,2,5
3,6039,3,5


## 4) CSR matrix를 직접 만들어 봅시다.


In [22]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [25]:
als_model = AlternatingLeastSquares(factors=100, regularization = 0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [26]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [27]:
als_model.fit(csr_data_transpose)

In [32]:
heemin, Toy_story = user_to_idx['heemin'], title_to_idx['Toy Story (1995)']
heemin_vector, Toy_story = als_model.user_factors[heemin], als_model.item_factors[Toy_story]

In [33]:
heemin_vector

array([ 0.14589363,  0.2682459 , -0.28842348,  0.16775829, -0.49477306,
       -0.27095786,  0.32666174, -0.42476076,  0.07823713,  0.37582463,
       -0.07541524, -0.61281455,  0.20023195,  0.09545047,  0.49294576,
       -0.39012828,  0.41398615, -0.79675364,  0.43076068, -0.4102762 ,
        0.20184022, -0.22278923, -0.22509311,  0.8219797 , -0.21643591,
        0.26026836,  0.02979103, -0.34966066, -0.13144678,  0.48885503,
        0.6962286 , -0.07342058, -0.00925786,  0.03219969, -0.16860521,
        0.24703011, -0.1167874 , -0.10241173,  0.44278306, -0.3285881 ,
        0.27035332, -0.44904244, -0.01952643, -0.10226033,  0.96008545,
        0.25152215, -0.41078624, -0.0871465 , -0.4211384 , -0.02450258,
        0.3039801 ,  0.05039906,  0.2024726 ,  0.05132294, -0.434391  ,
        0.13858066, -0.02961255, -0.06880128, -0.09550335,  0.2526058 ,
       -0.47488663, -0.11347625, -0.0405945 ,  0.09432876, -0.20963325,
        0.19126987, -0.06294941, -0.47079897,  0.28368118,  0.08

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [34]:
np.dot(heemin_vector, Toy_story)

0.47479156

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [35]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [38]:
get_similar_title('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


In [43]:
user = user_to_idx['heemin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

Toy Story 2 (1999) 0000.483
Babe (1995) 0000.307
Santa Clause, The (1994) 0000.294
Bug's Life, A (1998) 0000.281
Aladdin (1992) 0000.231
Grumpy Old Men (1993) 0000.221
Mask, The (1994) 0000.218
Hook (1991) 0000.204
Lion King, The (1994) 0000.183
Nutty Professor, The (1996) 0000.179
Nikita (La Femme Nikita) (1990) 0000.179
Home Alone (1990) 0000.177
Space Jam (1996) 0000.172
Sixth Sense, The (1999) 0000.172
Star Wars: Episode I - The Phantom Menace (1999) 0000.171
Six Days Seven Nights (1998) 0000.168
Hercules (1997) 0000.167
Beauty and the Beast (1991) 0000.166
Indian in the Cupboard, The (1995) 0000.165
Dragonheart (1996) 0000.165


## 마무리

Exploration 7 에서는 영화 추천 시스템에 대해서 알아보았다.
최근 넷플릭스나 왓챠 등에서 영화 추천 시스템을 많이 이용하고 있는데 해당 서비스가 어떻게 제공되고 있는지 알게 되는 과정을 갖게 되어서 흥미로웠다.